In [ ]:
PROJECT = "era5-land-project"

# Authorization and Imports

In [ ]:
!gcloud auth login --project {PROJECT}


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=XGYCzkYhXFlg2KdLZU38xKmRi3dWrd&prompt=consent&token_usage=remote&access_type=offline&code_challenge=OgYnA4L5QdCH1ROm-Ch1XgZWu2rFySfzQvI4FmsPNc4&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQlEd8xUHbGIXacM1qQI_XGMA32bax_bzhfSsgqO69cnASPnN0Wj7tdDTPuygFB9qhQVPw

You are now logged in as [szhanerz@gmail.com].
Your current project is 

In [ ]:
import datetime
import ee
import logging # to check task status when exporting images
import math


In [ ]:
ee.Authenticate()
ee.Initialize(project=PROJECT)

Function to filter 4-hourly intervals and add day properties to each image

# Efficient Aggregation of 4-Hourly Time Series into Daily Data Time Series

In [ ]:
def add_date_properties(image):
    date = ee.Date(image.get('system:time_start'))
    hour = date.get('hour')
    day_of_year = date.getRelative('day', 'year')
    return image.set('day_of_year', day_of_year).set('hour', hour)

Function to count the number of 4-hourly intervals per day

In [ ]:
def count_4hourly_intervals(year, start_day, end_day, region):
    start_date = ee.Date.fromYMD(year, 1, 1).advance(start_day - 1, 'day')
    end_date = ee.Date.fromYMD(year, 1, 1).advance(end_day, 'day')
    era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(start_date, end_date).filterBounds(region)

    # day properties to each image and filter to 4-hour intervals
    era_hourly = era_hourly.map(add_date_properties).filter(ee.Filter.eq('hour', 0)).merge(
        era_hourly.map(add_date_properties).filter(ee.Filter.eq('hour', 4))).merge(
        era_hourly.map(add_date_properties).filter(ee.Filter.eq('hour', 8))).merge(
        era_hourly.map(add_date_properties).filter(ee.Filter.eq('hour', 12))).merge(
        era_hourly.map(add_date_properties).filter(ee.Filter.eq('hour', 16))).merge(
        era_hourly.map(add_date_properties).filter(ee.Filter.eq('hour', 20)))

    daily_counts = era_hourly.aggregate_histogram('day_of_year')

    return daily_counts

add this into
- intermediary output

In [ ]:
year = 2023
start_day = 1
end_day = 10  # small range for demonstration purposes
region = ee.Geometry.Point([0, 0]).buffer(10000)  # random region

In [ ]:
daily_counts = count_4hourly_intervals(year, start_day, end_day, region)

daily_counts_info = daily_counts.getInfo()
print('Daily counts of 4-hourly intervals:', daily_counts_info)

Daily counts of 4-hourly intervals: {'0': 6, '1': 6, '2': 6, '3': 6, '4': 6, '5': 6, '6': 6, '7': 6, '8': 6, '9': 6}


In [ ]:
daily_counts_values = list(daily_counts_info.values())
print('First few values:', daily_counts_values[:5])
print('Max value:', max(daily_counts_values))
print('Min value:', min(daily_counts_values))
print('Mean value:', sum(daily_counts_values) / len(daily_counts_values))

First few values: [6, 6, 6, 6, 6]
Max value: 6
Min value: 6
Mean value: 6.0


# Computing Degree-Day Values from 4-Hourly Data

1. Helps track and group temperature data into 4-hour intervals.

In [ ]:
def add_date_properties(image):
    date = ee.Date(image.get('system:time_start'))
    hour = date.get('hour')
    day_of_year = date.getRelative('day', 'year')
    return image.set('day_of_year', day_of_year).set('hour', hour)

2. Calculates degree days relative to the base temperature (e.g., -20°C).

In [ ]:
def compute_4hourly_degree_days(image, base_temp=-20):
    temp_band = image.select('temperature_2m').subtract(273.15)  # Convert Kelvin to Celsius
    degree_days = temp_band.subtract(base_temp).max(0)
    return degree_days.rename(f'degree_days_above_{base_temp}')

In [ ]:
def aggregate_monthly_degree_days(year, month, base_temp, region):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(start_date, end_date).filterBounds(region)

    degree_days = era_hourly.map(lambda img: compute_4hourly_degree_days(img, base_temp))
    degree_days = degree_days.map(add_date_properties)

    # Sum degree days by day
    daily_sum = degree_days.reduce(ee.Reducer.sum())
    # Count the number of 4-hour intervals per day
    daily_count = degree_days.reduce(ee.Reducer.count())
    # Calculate the average degree days per day (to handle missing data)
    daily_avg = daily_sum.divide(daily_count).rename(f'daily_avg_degree_days_above_{base_temp}')

    # Sum the daily averages over the month
    monthly_sum = daily_avg.reduce(ee.Reducer.sum()).rename(f'monthly_sum_degree_days_above_{base_temp}')

    year_band = ee.Image.constant(year).rename('year').toFloat()
    month_band = ee.Image.constant(month).rename('month').toFloat()

    combined = monthly_sum.addBands(year_band).addBands(month_band)

    return combined

In [ ]:
def aggregate_monthly_data(year, month, band_name, operation):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(start_date, end_date).select(band_name)

    if operation == 'max':
        aggregated_data = era_hourly.max()
    elif operation == 'min':
        aggregated_data = era_hourly.min()
    elif operation == 'mean':
        aggregated_data = era_hourly.mean()
    elif operation == 'sum':
        aggregated_data = era_hourly.sum()
    else:
        raise ValueError(f"Unsupported operation: {operation}")

    # Convert temperature from Kelvin to Celsius if the band name includes 'temperature'
    if 'temperature' in band_name:
        aggregated_data = aggregated_data.subtract(273.15)

    return aggregated_data.set('system:time_start', start_date.millis())


In [ ]:
def generate_monthly_aggregates(years, base_temps):
    monthly_aggregates = []

    for year in years:
        for month in range(1, 13):
            monthly_max = aggregate_monthly_data(year, month, 'temperature_2m', 'max').rename('max_temp')
            monthly_min = aggregate_monthly_data(year, month, 'temperature_2m', 'min').rename('min_temp')
            monthly_mean = aggregate_monthly_data(year, month, 'temperature_2m', 'mean').rename('mean_temp')
            monthly_precip = aggregate_monthly_data(year, month, 'total_precipitation', 'sum').rename('total_precip')

            degree_days_list = []
            for base_temp in base_temps:
                degree_day = aggregate_monthly_degree_days(year, month, base_temp, coordinates).select(f'monthly_sum_degree_days_above_{base_temp}') # drop it to tidy up coordinates  - nothing computation
                degree_days_list.append(degree_day)

            degree_days_combined = ee.Image.cat(degree_days_list)

            year_band = ee.Image.constant(year).rename('year').toFloat()
            month_band = ee.Image.constant(month).rename('month').toFloat()

            combined = monthly_max \
                                  .addBands(monthly_min) \
                                  .addBands(monthly_mean) \
                                  .addBands(monthly_precip) \
                                  .addBands(degree_days_combined) \
                                  .addBands(year_band) \
                                  .addBands(month_band)

            monthly_aggregates.append(combined)
    return ee.ImageCollection(monthly_aggregates)

In [ ]:
coordinates = ee.FeatureCollection("projects/era5-land-project/assets/itrdb_locations_unique_with_duplicate_lat_lon_info")


In [ ]:
monthly_aggregates_ic = generate_monthly_aggregates([2023], list(range(-5,0)))

In [ ]:
print(monthly_aggregates_ic.size().getInfo())


12


In [ ]:
print(monthly_aggregates_ic.first().bandNames().getInfo())
# print(monthly_aggregates_ic.first().getInfo())

['max_temp', 'min_temp', 'mean_temp', 'total_precip', 'monthly_sum_degree_days_above_-5', 'monthly_sum_degree_days_above_-4', 'monthly_sum_degree_days_above_-3', 'monthly_sum_degree_days_above_-2', 'monthly_sum_degree_days_above_-1', 'year', 'month']


export piece

In [ ]:
def zonal_stats(ic, fc, params=None):
    _params = {
        'reducer': ee.Reducer.mean(),  # Change this as needed (e.g., sum, median)
        'scale': 1000,  # Adjust scale to your needs
        'crs': None,
        'bands': None,
        'bandsRename': None,
        'imgProps': None,
        'imgPropsRename': None,
        'datetimeName': 'datetime',
        'datetimeFormat': 'YYYY-MM-dd HH:mm:ss'
    }
    if params:
        for param in params:
            _params[param] = params[param] or _params[param]

    img_rep = ic.first()
    non_system_img_props = ee.Feature(None).copyProperties(img_rep).propertyNames()
    if not _params['bands']:
        _params['bands'] = img_rep.bandNames()
    if not _params['bandsRename']:
        _params['bandsRename'] = _params['bands']
    if not _params['imgProps']:
        _params['imgProps'] = non_system_img_props
    if not _params['imgPropsRename']:
        _params['imgPropsRename'] = _params['imgProps']

    def map_function(img):
        img = ee.Image(img.select(_params['bands'], _params['bandsRename']))
        img = img.set(_params['datetimeName'], img.date().format(_params['datetimeFormat']))
        img = img.set('timestamp', img.get('system:time_start'))
        props_from = ee.List(_params['imgProps']).cat(ee.List([_params['datetimeName'], 'timestamp']))
        props_to = ee.List(_params['imgPropsRename']).cat(ee.List([_params['datetimeName'], 'timestamp']))
        img_props = img.toDictionary(props_from).rename(props_from, props_to)
        fc_sub = fc.filterBounds(img.geometry())
        return img.reduceRegions(
            collection=fc_sub,
            reducer=_params['reducer'],
            scale=_params['scale'],
            crs=_params['crs']
        ).map(lambda f: f.set(img_props))

    results = ic.map(map_function).flatten()
    return results

In [ ]:
def export_data_by_years(years, base_temps, feature_collection, export_folder):
    for year in years:
        for month in range(1, 13):
            monthly_aggregates_ic = generate_monthly_aggregates([year], base_temps)

            # Apply zonal statistics to the monthly aggregates for this month
            params = {'reducer': ee.Reducer.mean(), 'scale': 1000}
            ptsERA5monthly = zonal_stats(monthly_aggregates_ic.filter(ee.Filter.calendarRange(month, month, 'month')), feature_collection, params)

            # Define the export parameters
            export_description = f"monthly_aggregates_export_{year}_{month:02d}_degree_days"
            export_params = {
                'collection': ptsERA5monthly,
                'description': export_description,
                'folder': export_folder,
                'fileFormat': 'CSV'
            }

            # Start the export task
            task = ee.batch.Export.table.toDrive(**export_params)
            task.start()
            print(f"Exporting data: {export_description} to Google Drive in folder {export_folder}")

In [ ]:
base_temps = list(range(-20, 36))
years = [2023]  # Adjust the years as needed
coordinates = ee.FeatureCollection("projects/era5-land-project/assets/itrdb_locations_unique_with_duplicate_lat_lon_info")



In [ ]:
export_folder = "export_era5_monthly_new"
export_data_by_years(years, base_temps, coordinates, export_folder)

Exporting data: monthly_aggregates_export_2023_01_degree_days to Google Drive in folder export_era5_monthly_new
Exporting data: monthly_aggregates_export_2023_02_degree_days to Google Drive in folder export_era5_monthly_new
Exporting data: monthly_aggregates_export_2023_03_degree_days to Google Drive in folder export_era5_monthly_new
Exporting data: monthly_aggregates_export_2023_04_degree_days to Google Drive in folder export_era5_monthly_new
Exporting data: monthly_aggregates_export_2023_05_degree_days to Google Drive in folder export_era5_monthly_new
Exporting data: monthly_aggregates_export_2023_06_degree_days to Google Drive in folder export_era5_monthly_new
Exporting data: monthly_aggregates_export_2023_07_degree_days to Google Drive in folder export_era5_monthly_new
Exporting data: monthly_aggregates_export_2023_08_degree_days to Google Drive in folder export_era5_monthly_new
Exporting data: monthly_aggregates_export_2023_09_degree_days to Google Drive in folder export_era5_mont